In [1]:
#Put introductory header here

In [2]:
#package imports

#standard os and sys (probably won't be used, but just in case)
import os,sys

#pandas, to handle the csv/json data
import pandas as pd

#matplotlib.pyplot to handle most plotting
import matplotlib.pyplot as plt

#seabons as potential plotting alternative, may scrap later if I don't use it
import seaborn as sns

In [ ]:
#read in the import file

#place your data file here within the double quotes
#it should either be a json or csv file, directly from the download for the nomination stats tool
#modifications could render the file useless here...
#default example points towards "data_files/"
my_stats_file = ""